In [8]:
import os
import json
import pandas as pd

In [9]:
results_path = os.path.join(os.getcwd(), "regioni", "sidarthe", "Italy")
experiments = [dir for dir in os.listdir(results_path) if os.path.isdir(os.path.join(results_path, dir))]

exps = {}
for exp_dir in experiments:
    json_path = os.path.join(results_path, exp_dir, "final.json")
    if os.path.exists(json_path):
        with open(json_path) as json_file:
            exps[exp_dir] = json.load(json_file)



In [10]:
validation_key = "rmse"

df = pd.DataFrame()
 
#for exp_name, exp in exps.items():

best_exp = max(exps, key=lambda k: exps[k]["val_risks"][validation_key])

print(f"Best exp name: {best_exp}")
print("Settings used: ")
with open(os.path.join(results_path, best_exp, "settings.json")) as settings_file:
    for line in settings_file.readlines():
        print(line, end='')

Best exp name: 4e2290fe-a03c-4d52-9bc6-98e4be7fd5f8
Settings used: 
{

    "region": "Italy",

    "initial_values": {

        "alpha": 0.6,

        "beta": 0.11,

        "gamma": 0.7,

        "delta": 0.11,

        "epsilon": 0.171,

        "theta": 0.371,

        "xi": 0.017,

        "eta": 0.125,

        "mu": 0.017,

        "nu": 0.027,

        "tau": 0.01,

        "lambda": 0.034,

        "kappa": 0.017,

        "zeta": 0.125,

        "rho": 0.034,

        "sigma": 0.017

    },

    "learning_rates": {

        "alpha": 1e-05,

        "beta": 1e-05,

        "gamma": 1e-05,

        "delta": 1e-05,

        "epsilon": 1e-05,

        "theta": 1e-07,

        "xi": 1e-05,

        "eta": 1e-05,

        "mu": 1e-05,

        "nu": 1e-05,

        "tau": 1e-07,

        "lambda": 1e-05,

        "kappa": 1e-05,

        "zeta": 1e-05,

        "rho": 1e-05,

        "sigma": 1e-05

    },

    "target_weights": {

        "d_weight": 1.0,

        "r_weight": 10.0,